In [30]:
#imports
import torch

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.metrics import accuracy_score

In [31]:
#definim transformarea imaginilor
img_size = 224

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),  # Resize images
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize (ResNet standard)
])

In [32]:
#definim directoarele:
train_dir = "D:\Facultate\Licenta\DatasetTrim\Train"
valid_dir = "D:\Facultate\Licenta\DatasetTrim\Validation"
test_dir = "D:\Facultate\Licenta\DatasetTrim\Test"

In [33]:
train_dataset = ImageFolder(root=train_dir, transform=transform)
valid_dataset = ImageFolder(root=valid_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

In [34]:
#Creating DataLoaders - efficiently load batches of images during training.
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [35]:
#Load Pretrained ResNet Model
model = models.resnet50(pretrained=True)

# Modify the classifier for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

#Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

c:\Users\ALex\anaconda3\envs\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ALex\anaconda3\envs\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\ALex\anaconda3\envs\myenv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [36]:
#Freeze pretrained parameters:
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True  # Train only the last layer

In [37]:
early_stop_patience = 5
no_improve_epochs = 0

writer = SummaryWriter()

num_epochs = 10
best_val_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/Train", loss, epoch)
        writer.flush()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    #Validation phase
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(valid_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    
    # Reduce LR if validation loss stops improving
    scheduler.step(avg_val_loss)

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve_epochs = 0
        torch.save(model.state_dict(), "best_resnet50.pth")
        print("✅ Model Saved!")
    else:
        no_improve_epochs += 1

    if no_improve_epochs >= early_stop_patience:
        print("⏹ Early stopping triggered!")
        break

writer.close()
print("Training complete!")

Epoch [1/10], Train Loss: 0.5842, Val Loss: 0.6357
✅ Model Saved!
Epoch [2/10], Train Loss: 0.4529, Val Loss: 0.5365
✅ Model Saved!
Epoch [3/10], Train Loss: 0.3941, Val Loss: 0.6159
Epoch [4/10], Train Loss: 0.3600, Val Loss: 0.4973
✅ Model Saved!
Epoch [5/10], Train Loss: 0.3317, Val Loss: 0.5423
Epoch [6/10], Train Loss: 0.3066, Val Loss: 0.4953
✅ Model Saved!
Epoch [7/10], Train Loss: 0.3131, Val Loss: 0.5698
Epoch [8/10], Train Loss: 0.2953, Val Loss: 0.5747
Epoch [9/10], Train Loss: 0.2837, Val Loss: 0.4808
✅ Model Saved!
Epoch [10/10], Train Loss: 0.2735, Val Loss: 0.5357
Training complete!


In [38]:
#Evaluation on the Test set
model.load_state_dict(torch.load("best_resnet50.pth"))
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        preds = torch.sigmoid(outputs) > 0.5
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy after fine-tuning: {accuracy * 100:.2f}%")

C:\Users\ALex\AppData\Local\Temp\ipykernel_19152\2458586844.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_resnet50.pth"))


Test Accuracy after fine-tuning: 71.58%
